In [ ]:
%matplotlib notebook
from tensorforce.environments import Environment
from tensorforce.agents import Agent
from tensorforce.execution import Runner
import random
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnnotationBbox, OffsetImage
from PIL import Image
from matplotlib.patches import Rectangle
from matplotlib import animation

class CustomEnvironment(Environment):
    
    def __init__(self):
        super().__init__()
        self.numMoves = 0
        self.goalx = 95
        self.goaly = 95
        self.envStates = [0,0,1,1]
    
    def states(self):
        return dict(type='float', shape=(4,))
    
    def actions(self):
        return dict(type='float', shape=(2,), min_value = -10, max_value = 10)
    
    def max_episode_timesteps(self):
        return super().max_episode_timesteps()
    
    def reset(self):
        #velocityx, velocityy, positionx, positiony
        state = [0, 0, 85, 85]
        return state
    
    def close(self):
        super().close()
    
    def execute(self, actions):
        reward = -0.25
        time_step = 0.1
        x = self.envStates[2]
        y = self.envStates[3]
        xvel = self.envStates[0]
        yvel = self.envStates[1]
        xacc = actions[0]
        yacc = actions[1]
        xvel = xvel + time_step * xacc
        yvel = yvel + time_step * yacc
        x = x + time_step * xvel
        y = y + time_step * yvel
        newState = [xvel, yvel, x, y]
        self.increaseNumMoves()
        if(isRobotCrossingWalls(self.envStates[2], self.envStates[3], newState[2], newState[3])):
            reward = reward - 100
            newState=[0, 0, self.envStates[2], self.envStates[3]]
            
            
        successReward, terminal = self.checkTerminal(newState[2], newState[3])
        reward = reward + successReward
        return newState, terminal, reward
    
    def increaseNumMoves(self):
        self.numMoves = self.numMoves + 1
    
    def checkTerminal(self,x1,y1):
        reward = 0
        terminal = False
        x2 = self.goalx
        y2 = self.goaly
        if((((y2-y1)**2)+((x2-x1)**2))**0.5 <= 5):
            reward = 10
            terminal = True
        return reward, terminal
        

class Wall:
    p1x = 0
    p1y = 0
    p2x = 0
    p2y = 0
    numIntersect = 0

numWalls = 9

class Robot:
    x_velocity = 0
    y_velocity = 0
    x_position = 0
    y_position = 0

class robotState:
    x_velocity = 0
    y_velocity = 0
    x_position = 1
    y_position = 1
    reward = 0

class Action:
    x_acceleration = 3
    y_acceleration = 3

robot = Robot()


environment = Environment.create(
    environment=CustomEnvironment, max_episode_timesteps=1000
)

#agentPPO = Agent.create(
    #agent='ppo', environment=environment, batch_size=10, learning_rate=1e-3
#)

#agentPPO = Agent.create(
    #agent='ppo', environment=environment, batch_size=5, predict_terminal_values=False, learning_rate=1e-3
#)

#agentTensor = Agent.create(
    #agent='tensorforce', environment=environment, update=64,
    #optimizer=dict(optimizer='adam', learning_rate=1e-3),
    #objective='policy_gradient', reward_estimation=dict(horizon=20)
#)
agentPPO = Agent.create(agent='ppo.json', environment=environment)
agentTensor = Agent.create(agent='tensorforce.json', environment=environment)

robotAction = Action()

wallTop = Wall()
wallBottom = Wall()
wallLeft = Wall()
wallRight = Wall()
wallLeft.p1x=0;wallLeft.p1y=0;wallLeft.p2x=0;wallLeft.p2y=100
wallTop.p1x=0;wallTop.p1y=100;wallTop.p2x=100;wallTop.p2y=100
wallRight.p1x=100;wallRight.p1y=100;wallRight.p2x=100;wallRight.p2y=0
wallBottom.p1x=100;wallBottom.p1y=0;wallBottom.p2x=0;wallBottom.p2y=0

stateHistoryPPO = []
stateHistoryTensor = []
#stateHistory[0].x_velocity = 0
#stateHistory[0].x_velocity = 0
#stateHistory[0].x_position = 1
#stateHistory[0].y_position = 1

wallArray = [wallLeft,wallTop,wallRight,wallBottom]

for i in range(4,numWalls):
    wallArray.append(Wall())

fig, ax = plt.subplots()
ax.set_xlim([-50,150])
ax.set_ylim([-50,150])

patchPPO = plt.Circle((85, 85), 1.5, fc='g')
patchTensor = plt.Circle((85, 85), 1.5, fc='b')
goal = plt.Circle((95, 95), 5, fc='m')

ax.plot([wallLeft.p1x,wallLeft.p2x],[wallLeft.p1y,wallLeft.p2y],color='b')
ax.plot([wallTop.p1x,wallTop.p2x],[wallTop.p1y,wallTop.p2y],color='b')
ax.plot([wallRight.p1x,wallRight.p2x],[wallRight.p1y,wallRight.p2y],color='b')
ax.plot([wallBottom.p1x,wallBottom.p2x],[wallBottom.p1y,wallBottom.p2y],color='b')
minRangeFirstPoint = -20
maxRangeFirstPoint = 120
minRangeSecondPoint = 20
maxRangeSecondPoint = 80

def plotMaze():
    for i in range(4,numWalls):
        ax.plot([wallArray[i].p1x,wallArray[i].p2x],[wallArray[i].p1y,wallArray[i].p2y],color='r')
    ax.add_patch(patchPPO)
    ax.add_patch(patchTensor)
    ax.add_patch(goal)
    plt.show
    
def randomMaze():
    for i in range(4,numWalls):
        wallArray[i].numIntersect = 0
        wallArray[i].p1x = random.randint(minRangeFirstPoint,maxRangeFirstPoint)
        wallArray[i].p1y = random.randint(minRangeFirstPoint,maxRangeFirstPoint)
        wallArray[i].p2x = random.randint(minRangeSecondPoint,maxRangeSecondPoint)
        wallArray[i].p2y = random.randint(minRangeSecondPoint,maxRangeSecondPoint)

def isSolvable():
    for i in range(4, numWalls):
        for j in range(numWalls): 
            if(i!=j and isIntersection(wallArray[i].p1x,wallArray[i].p1y,wallArray[i].p2x,wallArray[i].p2y,wallArray[j].p1x,wallArray[j].p1y,wallArray[j].p2x,wallArray[j].p2y)):
                wallArray[i].numIntersect=(wallArray[i].numIntersect)+1
        if(wallArray[i].numIntersect>1):
            return False
    return True

def isIntersection(x1,y1,x2,y2,x3,y3,x4,y4):
    
    if(x2==x1 and x4==x3):
        return False
    elif(x2==x1 and y3==y4):
        intersectionX=x1
        intersectionY=y3
        if(y1>y2):
            if(intersectionY>=y2 and intersectionY<=y1):
                if(x3>x4):
                    if(intersectionX>=x4 and intersectionX<=x3):
                        return True
                if(x4>x3):
                    if(intersectionX>=x3 and intersectionX<=x4):
                        return True
        if(y2>y1):
            if(intersectionY>=y1 and intersectionY<=y2):
                if(x3>x4):
                    if(intersectionX>=x4 and intersectionX<=x3):
                        return True
                if(x4>x3):
                    if(intersectionX>=x3 and intersectionX<=x4):
                        return True
        return False
    elif(x4==x3 and y2==y1):
        intersectionX=x3
        intersectionY=y2
        if(y3>y4):
            if(intersectionY>=y4 and intersectionY<=y3):
                if(x2>x1):
                    if(intersectionX>=x1 and intersectionX<=x2):
                        return True
                if(x1>x2):
                    if(intersectionX>=x2 and intersectionX<=x1):
                        return True
        if(y4>y3):
            if(intersectionY>=y3 and intersectionY<=y4):
                if(x2>x1):
                    if(intersectionX>=x1 and intersectionX<=x2):
                        return True
                if(x1>x2):
                    if(intersectionX>=x2 and intersectionX<=x1):
                        return True
        return False
    elif(x2==x1):
        m2=(y4-y3)/(x4-x3)
        intersectionX=x1
        b2=y3-(m2*x3)
        intersectionY=(m2*intersectionX)+b2
        if(y2>y1):
            if(intersectionY<=y2 and intersectionY>=y1):
                if(y3>y4):
                    if(intersectionY<=y3 and intersectionY>=y4):
                        return True
                if(y4>y3):
                    if(intersectionY<=y4 and intersectionY>=y3):
                        return True
        if(y1>y2):
            if(intersectionY<=y1 and intersectionY>=y2):
                if(y3>y4):
                    if(intersectionY<=y3 and intersectionY>=y4):
                        return True
                if(y4>y3):
                    if(intersectionY<=y4 and intersectionY>=y3):
                        return True
        return False
    elif(x4==x3):
        m1=(y2-y1)/(x2-x1)
        b1=y1-(m1*x1)
        intersectionX=x3
        intersectionY=(m1*intersectionX)+b1
        if(y3>y4):
            if(intersectionY<=y3 and intersectionY>=y4):
                if(y1>y2):
                    if(intersectionY<=y1 and intersectionY>=y2):
                        return True
                if(y2>y1):
                    if(intersectionY<=y2 and intersectionY>=y1):
                        return True
        if(y4>y3):
            if(intersectionY<=y4 and intersectionY>=y3):
                if(y1>y2):
                    if(intersectionY<=y1 and intersectionY>=y2):
                        return True
                if(y2>y1):
                    if(intersectionY<=y2 and intersectionY>=y1):
                        return True
        return False
    else:
        m1=(y2-y1)/(x2-x1)
        m2=(y4-y3)/(x4-x3)
        if(m1!=m2):
            b1=y1-(m1*x1)
            b2=y3-(m2*x3)
            intersectionX=(b2-b1)/(m1-m2)
            if(x1>x2):
                if(intersectionX<=x1 and intersectionX>=x2):
                    if(x3>x4):
                        if(intersectionX<=x3 and intersectionX>=x4):
                            return True
                    if(x4>x3):
                        if(intersectionX<=x4 and intersectionX>=x3):
                            return True
            if(x2>x1):
                if(intersectionX<=x2 and intersectionX>=x1):
                    if(x3>x4):
                        if(intersectionX<=x3 and intersectionX>=x4):
                            return True
                    if(x4>x3):
                        if(intersectionX<=x4 and intersectionX>=x3):
                            return True
        return False


def printWallCoordinates():
    for i in range(4,numWalls):
        print("Red wall", i-3, end = ': ')
        print("Intersection =", wallArray[i].numIntersect, end = ', ')
        print("x1 =", wallArray[i].p1x, end = ', ')
        print("y1 =", wallArray[i].p1y, end = ', ')
        print("x2 =", wallArray[i].p2x, end = ', ')
        print("y2 =", wallArray[i].p2y)

#def plotRobot():
    
#def moveRobotDiagonal():


def isRobotCrossingWalls(rx1, ry1, rx2, ry2):
    for i in range(numWalls):
        if(isIntersection(rx1, ry1, rx2, ry2, wallArray[i].p1x, wallArray[i].p1y, wallArray[i].p2x, wallArray[i].p2y)):
            return True
    return False
    
    

   
while(True):
    randomMaze()
    if(isSolvable()):
        break 

plotMaze()
#moveRobotDiagonal()

#printWallCoordinates()
def printStates(stateHistory, agentName):
    success = 0
    for i in range(len(stateHistory)):
        print("State", i,"x_position is",str(stateHistory[i].x_position) + ".")
        print("State", i,"y_position is",str(stateHistory[i].y_position) + ".")
        print("State", i,"x_velocity is",str(stateHistory[i].x_velocity) + ".")
        print("State", i,"y_velocity is",str(stateHistory[i].y_velocity) + ".")
        print("State", i,"reward is",str(stateHistory[i].reward) + ".")
        if(stateHistory[i].reward == 9.75):
            success += 1
        print()
    percent = (success/episodes)*100
    print(agentName,"reached the goal",success,"times.")
    print()
    print(agentName,"reached the goal",str(percent) + "%","of the times.")
    print()
    


#100 episodes for PPO Agent
episodes = 100
for _ in range(episodes):
    originalState = robotState()
    
    originalState.x_velocity = 0
    originalState.y_velocity = 0
    originalState.x_position = 85
    originalState.y_position = 85
    originalState.reward = 0
    stateHistoryPPO.append(originalState)
    environment.envStates = environment.reset()
    terminal = False
    while not terminal:
        actions = agentPPO.act(states=environment.envStates)
        environment.envStates, terminal, reward = environment.execute(actions)
        agentPPO.observe(terminal=terminal, reward=reward)
        state = robotState()
        state.x_velocity = environment.envStates[0]
        state.y_velocity = environment.envStates[1]
        state.x_position = environment.envStates[2]
        state.y_position = environment.envStates[3]
        state.reward = reward
        stateHistoryPPO.append(state)


#100 episodes for Tensorforce Agent        
for _ in range(episodes):
    originalState = robotState()
    
    originalState.x_velocity = 0
    originalState.y_velocity = 0
    originalState.x_position = 85
    originalState.y_position = 85
    originalState.reward = 0
    stateHistoryTensor.append(originalState)
    environment.envStates = environment.reset()
    terminal = False
    while not terminal:
        actions = agentTensor.act(states=environment.envStates)
        environment.envStates, terminal, reward = environment.execute(actions)
        agentTensor.observe(terminal=terminal, reward=reward)
        state = robotState()
        state.x_velocity = environment.envStates[0]
        state.y_velocity = environment.envStates[1]
        state.x_position = environment.envStates[2]
        state.y_position = environment.envStates[3]
        state.reward = reward
        stateHistoryTensor.append(state)

def robotAnimation(i):
    if(i <= len(stateHistoryPPO)):
        x = stateHistoryPPO[i-1].x_position
        y = stateHistoryPPO[i-1].y_position
        patchPPO.center = (x, y)
    else:
        x = stateHistoryPPO[len(stateHistoryPPO)-1].x_position
        y = stateHistoryPPO[len(stateHistoryPPO)-1].x_position
        patchPPO.center(x, y)
    
    if(i <= len(stateHistoryTensor)):
        x = stateHistoryTensor[i-1].x_position
        y = stateHistoryTensor[i-1].y_position
        patchTensor.center = (x, y)
    else:
        x = stateHistoryTensor[len(stateHistoryTensor)-1].x_position
        y = stateHistoryTensor[len(stateHistoryTensor)-1].x_position
        patchTensor.center(x, y)
    
    return patchPPO, patchTensor

if(len(stateHistoryPPO) > len(stateHistoryTensor)):
    anim = animation.FuncAnimation(fig, robotAnimation, frames=len(stateHistoryPPO), interval=50, repeat=False)
else:
    anim = animation.FuncAnimation(fig, robotAnimation, frames=len(stateHistoryTensor), interval=50, repeat=False)
    
    
printStates(stateHistoryPPO, "PPO")
printStates(stateHistoryTensor, "Tensorforce")

plt.show()


